### **BERT CODE - FIRST RUN (UNKNOWN TEST AND CSV EMBEDDINGS)**


In [115]:
!pip3 install transformers

^C


In [116]:
!pip3 install PyPDF2

  Using cached transformers-4.29.2-py3-none-any.whl (7.1 MB)
     ------------------------------------ 236.8/236.8 kB 763.0 kB/s eta 0:00:00
  Using cached numpy-1.24.3.tar.gz (10.9 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  Getting requirements to build wheel did not run successfully.
  exit code: 1
  
  [40 lines of output]
  Traceback (most recent call last):
    File "C:\msys64\mingw64\lib\python3.10\site-packages\pip\_vendor\pep517\in_process\_in_process.py", line 351, in <module>
      main()
    File "C:\msys64\mingw64\lib\python3.10\site-packages\pip\_vendor\pep517\in_process\_in_process.py", line 333, in main
      json_out['return_val'] = hook(**hook_input['kwargs'])
    File "C:\msys64\mingw64\lib\python3.10\site-packages\pip\_vendor\pep517\in_process\_in_process.py", line 112, in get_requires_for_build_wheel
      backend = _build_backend()
    File "C:\msys64\mingw64\lib\python3.10\site-packages\pip\_vendor\pep517\in_process\_in_process.py", line 77, in _build_backend
      obj = import_module(mod_path)
    File "C:\msys64\mingw64\lib\python3.10\importlib\__init__.py", line 126, in import_module
      return _bootstrap._gcd_import(name[level:], packag

  Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: C:\msys64\mingw64\bin\python.exe -m pip install --upgrade pip


In [117]:
!pip3 install nltk

^C


In [120]:
import nltk 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Shashwat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [121]:
from transformers import BertTokenizer, BertModel
import PyPDF2
import re
import unicodedata
from nltk.corpus import stopwords
import pandas as pd
import torch
import numpy as np

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

pdf_path = "C:\\Users\\Shashwat\\Desktop\\Shashu\\SEM 6\\PR 301- Review of research papers\\OmPrakash Kaiwartya\\A Dynamic Congestion Control Scheme for safety applications.pdf"
csv_path = "C:\\Users\\Shashwat\\Desktop\\Shashu\\SEM 6\\Project Files\\Lemmatized_Output.csv"
text_column = "PDF"
k = 5

stop_words = set(stopwords.words('english'))


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


**PREPROCESSING**

In [122]:
def preprocess_text(text):
    if not isinstance(text, str):
        return ''
    text = text.replace('\n', ' ').replace('\r', ' ')
    text = re.sub('<[^>]*>', '', text)
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    text = re.sub('http[s]?://\S+', '', text)
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.split()
    text = [word for word in text if word not in stop_words]
    text = [word for word in text if len(word) > 2]
    text = ' '.join(text)
    return text

def preprocess_test_document(pdf_path):
    test_document_text = extract_text_from_pdf(pdf_path)
    preprocessed_text = preprocess_text(test_document_text)
    return preprocessed_text

def preprocess_csv_file(csv_path, text_column):
    data = pd.read_csv(csv_path)
    csv_text_data = data[text_column].tolist()
    preprocessed_texts = [preprocess_text(text) for text in csv_text_data]
    return preprocessed_texts


**TEXT EXTRACTION FOR TEST PDF**


In [123]:
def extract_text_from_pdf(pdf_path):
    try:
        with open(pdf_path, 'rb') as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            pdf_text = ''
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                pdf_text += page.extract_text()
    except KeyError as e:
        print(f"Error processing file '{pdf_path}': {e}")
        return ''
    return pdf_text


**ENCODING**

In [124]:
def encode_documents(documents):
    encoded_texts = tokenizer(documents, padding=True, truncation=True, return_tensors='pt')
    return encoded_texts

**EMBEDDINGS**

In [125]:

def generate_embeddings(encoded_texts):
    with torch.no_grad():
        model.eval()
        input_ids = encoded_texts['input_ids']
        attention_mask = encoded_texts['attention_mask']
        output = model(input_ids=input_ids, attention_mask=attention_mask)
        embeddings = output.last_hidden_state[:, 0, :]
    return embeddings

**SIMILARITY**

In [126]:
def calculate_similarity(test_embedding, csv_embeddings):
    similarity_scores = []
    indices = []
    for i, csv_embedding in enumerate(csv_embeddings):
        similarity = torch.cosine_similarity(test_embedding, csv_embedding)
        similarity_scores.append(similarity.item())
        indices.append(i)
    return csv_embeddings, similarity_scores, indices

**RETRIEVAL**

In [127]:
def retrieve_top_k_unique_authors(csv_data, similarity_scores, indices, k):
    sorted_indices = sorted(range(len(similarity_scores)), key=lambda i: similarity_scores[i], reverse=True)
    top_k_indices = sorted_indices[:k]
    top_k_indices = np.array(indices)[top_k_indices]  # Convert indices to NumPy array
    top_k_authors = csv_data.iloc[top_k_indices]['AUTHOR'].unique()
    return top_k_authors

In [128]:
# Batch size for batch processing
batch_size = 100

# Preprocess the test PDF document
preprocessed_test_document = preprocess_test_document(pdf_path)

# Preprocess the CSV file
preprocessed_csv_texts = preprocess_csv_file(csv_path, text_column)

In [129]:
# Encode the test document and CSV document text
encoded_test_document = encode_documents([preprocessed_test_document])

encoded_csv_texts = encode_documents(preprocessed_csv_texts)

In [130]:
# Generate embeddings for the test document
test_embedding = generate_embeddings(encoded_test_document) 

# Save the test embedding to a file
np.save("test_embedding.npy", test_embedding.numpy())

# Load the test embedding from the file
test_embedding = torch.from_numpy(np.load("test_embedding.npy"))


In [ ]:
# Generate embeddings for the CSV file in batches
csv_embeddings = []
num_batches = int(np.ceil(len(encoded_csv_texts.input_ids) / batch_size))
for i in range(num_batches):
    batch_inputs = {
        'input_ids': encoded_csv_texts.input_ids[i * batch_size:(i + 1) * batch_size],
        'attention_mask': encoded_csv_texts.attention_mask[i * batch_size:(i + 1) * batch_size]
    }
    batch_embeddings = generate_embeddings(batch_inputs)
    csv_embeddings.append(batch_embeddings)

# Concatenate the embeddings from different batches
csv_embeddings = torch.cat(csv_embeddings)

# Save the CSV embeddings to a file
np.save("csv_embeddings2.npy", csv_embeddings.numpy())

In [ ]:
# Calculate similarity between the test document and CSV documents
csv_embeddings, similarity_scores, indices = calculate_similarity(test_embedding, csv_embeddings)

# Load the CSV data
csv_data = pd.read_csv(csv_path)

# Retrieve top-k unique authors
top_k_authors = retrieve_top_k_unique_authors(csv_data, similarity_scores, indices, k)

# Print the top-k authors
print(top_k_authors.tolist())

# **BERT CODE -(NEW TEST DOC AND EXISTING CSV EMBEDDINGS)**

In [134]:
from transformers import BertTokenizer, BertModel
import PyPDF2
import re
import unicodedata
from nltk.corpus import stopwords
import pandas as pd
import torch
import numpy as np

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

pdf_path = "C:\\Users\\Shashwat\\Desktop\\Shashu\\SEM 6\\PR 301- Review of research papers\\OmPrakash Kaiwartya\\A Dynamic Congestion Control Scheme for safety applications.pdf"
csv_path = "C:\\Users\\Shashwat\\Desktop\\Shashu\\SEM 6\\Project Files\\Lemmatized_Output.csv"
text_column = "PDF"
k = 10

stop_words = set(stopwords.words('english'))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


 **FUNCTION DEFINING**

In [147]:
def preprocess_test_document(pdf_path):
    test_document_text = extract_text_from_pdf(pdf_path)
    preprocessed_text = preprocess_text(test_document_text)
    return preprocessed_text

def preprocess_csv_file(csv_path, text_column):
    data = pd.read_csv(csv_path)
    csv_text_data = data[text_column].tolist()
    preprocessed_texts = [preprocess_text(text) for text in csv_text_data]
    return preprocessed_texts

def encode_documents(documents):
    encoded_texts = tokenizer(documents, padding=True, truncation=True, return_tensors='pt')
    return encoded_texts

def generate_embeddings(encoded_texts):
    with torch.no_grad():
        model.eval()
        input_ids = encoded_texts['input_ids']
        attention_mask = encoded_texts['attention_mask']
        output = model(input_ids=input_ids, attention_mask=attention_mask)
        embeddings = output.last_hidden_state[:, 0, :]
    return embeddings

def calculate_similarity(test_embedding, csv_embeddings):
    similarity_scores = []
    indices = []
    for i, csv_embedding in enumerate(csv_embeddings):
        similarity = torch.cosine_similarity(test_embedding, csv_embedding)
        similarity_scores.append(similarity.item())
        indices.append(i)
    return csv_embeddings, similarity_scores, indices

def retrieve_top_k_unique_authors(csv_data, similarity_scores, indices, k):
    sorted_indices = sorted(range(len(similarity_scores)), key=lambda i: similarity_scores[i], reverse=True)
    top_k_indices = sorted_indices[:k]
    top_k_indices = np.array(indices)[top_k_indices]  # Convert indices to NumPy array
    top_k_authors = csv_data.iloc[top_k_indices]['AUTHOR'].unique()
    return top_k_authors

In [146]:
# Preprocess the test PDF document
preprocessed_test_document = preprocess_test_document(pdf_path)

In [145]:
# Encode the test document and CSV document text
encoded_test_document = encode_documents([preprocessed_test_document])

In [144]:
# Generate embeddings for the test document
test_embedding = generate_embeddings(encoded_test_document) 

# Save the test embedding to a file
np.save("test_embedding.npy", test_embedding.numpy())

# Load the test embedding from the file
test_embedding = torch.from_numpy(np.load("test_embedding.npy"))

In [143]:
csv_embeddings = torch.from_numpy(np.load("csv_embeddings_final.npy")) 

In [148]:
def retrieve_top_k_unique_authors(csv_data, similarity_scores, indices, k):
    sorted_indices = np.argsort(similarity_scores)[::-1]  # Sort indices in descending order
    valid_indices = [idx for idx in sorted_indices if idx < len(csv_data)]  # Filter out-of-bounds indices
    top_k_indices = valid_indices[:k]  # Select top-k valid indices
    top_k_authors = csv_data.iloc[top_k_indices]['AUTHOR'].unique()
    return top_k_authors

# Calculate similarity between the test document and CSV documents
csv_embeddings, similarity_scores, indices = calculate_similarity(test_embedding, csv_embeddings)

# Load the CSV data
csv_data = pd.read_csv(csv_path)

# Retrieve top-k unique authors
top_k_authors = retrieve_top_k_unique_authors(csv_data, similarity_scores, indices, k)

# Print the top-k authors
print(top_k_authors.tolist())


['Tandra Pal', 'Dr.Rohit Beniwal', 'Barsha Mitra', 'Prof. B Subudhi', 'Tingwen Huang', 'Aruna Malapati', 'Dilip Singh Sisodia', 'Amit Saxena']
